# How to use `twobit`

In [1]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, execute, compile, IBMQ, BasicAer
import numpy as np
import random

def get_backend(device):
    """Returns backend object for device specified by input string."""
    try:
        backend = BasicAer.get_backend(device)
    except:
        print("You are using an IBMQ backend. The results for this are provided in accordance with the IBM Q Experience EULA.\nhttps://quantumexperience.ng.bluemix.net/qx/terms") # Legal stuff! Yay!
        backend = IBMQ.get_backend(device)
    return backend

class twobit:
    """An object that can store a single boolean value, but can do so in two incompatible ways. It is implemented on a single qubit using two complementary measurement bases."""
    
    def __init__(self):
        """Create a twobit object, initialized to give a random boolean value for both measurement types."""
        self.qr = QuantumRegister(1)
        self.cr = ClassicalRegister(1)
        self.qc = QuantumCircuit(self.qr, self.cr)
        self.prepare({'Y':None})
        
    def prepare(self,state):
        """Supplying `state={basis,b}` prepares a twobit with the boolean `b` stored using the measurement type specified by `basis` (which can be 'X', 'Y' or 'Z').
        
        Note that `basis='Y'` (and arbitrary `b`) will result in the twobit giving a random result for both 'X' and 'Z' (and similarly for any one versus the remaining two). """
        self.qc = QuantumCircuit(self.qr, self.cr)
        if 'Y' in state:
            self.qc.h(self.qr[0])
            if state['Y']:
                self.qc.sdg(self.qr[0])
            else:
                self.qc.s(self.qr[0])
        elif 'X' in state:
            if state['X']:
                self.qc.x(self.qr[0])
            self.qc.h(self.qr[0])
        elif 'Z' in state:
            if state['Z']:
                self.qc.x(self.qr[0])
                
    def value (self,basis,device='qasm_simulator',noisy=False,shots=1024,mitigate=True):
        """Extracts the boolean value for the given measurement type. The twobit is also reinitialized to ensure that the same value would if the same call to `measure()` was repeated.
        
        basis = 'X' or 'Z', specifying the desired measurement type.
        device = A string specifying a backend. The noisy behaviour from a real device will result in some randomness in the value given, even if it has been set to a definite value for a given measurement type. This effect can be reduced using `mitigate=True`.
        shots = Number of shots used when extracting results from the qubit. A value of greater than 1 only has any effect for `mitigate=True`, in which case larger values of `shots` allow for better mitigation.
        mitigate = Boolean specifying whether mitigation should be applied. If so the values obtained over `shots` samples are considered, and the fraction which output `True` is calculated. If this is more than 90%, measure will return `True`. If less than 10%, it will return `False`, otherwise it returns a random value using the fraction as the probability."""
        if basis=='X':
            self.qc.h(self.qr[0])
        elif basis=='Y':
            self.qc.sdg(self.qr[0])
            self.qc.h(self.qr[0])
        self.qc.barrier(self.qr)
        
        ######################changes begin here######################
        if noisy==True:
            theta = 0.1*np.pi
        else:
            theta = noisy*np.pi
        if noisy:
            self.qc.rx(theta,self.qr)
            self.qc.ry(theta,self.qr)
            self.qc.rz(theta,self.qr)
        ######################changes end here######################
        
        self.qc.measure(self.qr,self.cr)
        try:
            job = execute(self.qc, backend=get_backend(device), noise_model=get_noise(noisy), shots=shots)
        except:
            job = execute(self.qc, backend=get_backend(device), shots=shots)
        stats = job.result().get_counts()
        if '1' in stats:
            p = stats['1']/shots
        else:
            p = 0
        if mitigate: # if p is close to 0 or 1, just make it 0 or 1
            if p<0.1:
                p = 0
            elif p>0.9:
                p = 1
        measured_value = ( p>random.random() )
        self.prepare({basis:measured_value})
        
        return measured_value

    def X_value (self,device='qasm_simulator',noisy=False,shots=1024,mitigate=True):
        """Extracts the boolean value via the X basis. For details of kwargs, see `value()`."""
        return self.value('X',device=device,noisy=noisy,shots=shots,mitigate=mitigate)

    def Y_value (self,device='qasm_simulator',noisy=False,shots=1024,mitigate=True):
        """Extracts the boolean value via the X basis. For details of kwargs, see `value()`."""
        return self.value('Y',device=device,noisy=noisy,shots=shots,mitigate=mitigate)
        
    def Z_value (self,device='qasm_simulator',noisy=False,shots=1024,mitigate=True):
        """Extracts the boolean value via the X basis. For details of kwargs, see `value()`."""
        return self.value('Z',device=device,noisy=noisy,shots=shots,mitigate=mitigate)
    

A qubit is the quantum version of a bit. So you can use it to store boolean values. That's basically what a `twobit` object does.

In [2]:
b = twobit()

We can prepare our bit with the value `True` or `False`, and then read it out again using the `Z_value()` method.

In [3]:
b.prepare({'Z':True})
print("    bit value =",b.Z_value() )

    bit value = True


In [4]:
b.prepare({'Z':False})
print("    bit value =",b.Z_value() )

    bit value = False


You probably notice the `Z` in all the lines above. This is because, though you can only store one bit in a single qubit, there are many ways to do it. So when preparing the bit, and when reading out the value, you need to specify what method is used. In the above, we used what is known as the `Z` basis.

The `twobit` object also supports the use of the so-called `X` basis.

In [5]:
b.prepare({'X':True})
print("    bit value =",b.X_value() )

    bit value = True


In [6]:
b.prepare({'X':False})
print("    bit value =",b.X_value() )

    bit value = False


These two ways of storing a bit are completely incompatible. If you encode using the `Z` basis and read out using the `X` (or vice-versa) you'll get a random result.

In [7]:
print("    Here are 10 trials with, each with True encoded in the Z basis. The values read out with X are:\n")
for trial in range(1,11):
    b.prepare({'Z':True})
    message = "        Try " + str(trial)+": "
    message += str( b.X_value() ) 
    print( message )

    Here are 10 trials with, each with True encoded in the Z basis. The values read out with X are:

        Try 1: False
        Try 2: False
        Try 3: True
        Try 4: False
        Try 5: True
        Try 6: False
        Try 7: True
        Try 8: False
        Try 9: True
        Try 10: False


Once you read a value out for a given basis, the qubit forgets anything that was encoded within it before the readout. So though encoding with `Z` and then reading out with `X` gives a random value, that value will remain if you read out using `X` again.

Below we do the same as before, but this time the readout is done 5 times during each individual trail, instead of just once.

In [8]:
for trial in range(1,11):
    message = "        Try " + str(trial)+": "
    b.prepare({'Z':True})
    for repeat in range(5):
        message += str( b.X_value() ) + ", "
    print(message)

        Try 1: True, True, True, True, True, 
        Try 2: True, True, True, True, True, 
        Try 3: False, False, False, False, False, 
        Try 4: True, True, True, True, True, 
        Try 5: True, True, True, True, True, 
        Try 6: False, False, False, False, False, 
        Try 7: False, False, False, False, False, 
        Try 8: True, True, True, True, True, 
        Try 9: False, False, False, False, False, 
        Try 10: False, False, False, False, False, 


This behaviour is exactly why `Z_value()` and `X_value()` need to be methods of the objects rather than just attributes. If they were attributes, it would imply that they can both have well defined values at the same time, which we can just look at whenever we want without changing the object. But this is not the case. Instead, the action of extracting the values requires the object to run a process, known as measurement, which can change what is going on inside the object. That's why it needs a method.

The `X_value()`, `Z_value()` and `value()` methods all have the standard kwargs `device`, `noisy` and `shots` as explained in [the README](README.md). When noise is present, the wrong output might be returned with a small probability. Some mitigation is done to make this less likely. This can be turned off by setting the `mitigate=True` qwarg to `False`. Large values of `shots` (such as 1024) allow the mitigation to work better than for smaller values. At the extreme of `shots=1`, the mitigation becomes powerless.

For example, here is the same setup as above (10 samples, each with 5 repeated readouts) but with unmitigated noise.

In [9]:
b = twobit()
for trial in range(1,11):
    message = "        Try " + str(trial)+": "
    b.prepare({'Z':True})
    for repeat in range(5):
        message += str( b.X_value(noisy=0.1,mitigate=False) ) + ", "
    print(message)

        Try 1: True, True, False, False, False, 
        Try 2: False, False, False, False, False, 
        Try 3: True, True, True, True, True, 
        Try 4: False, False, False, False, False, 
        Try 5: True, True, True, True, True, 
        Try 6: True, True, True, True, True, 
        Try 7: True, True, True, True, True, 
        Try 8: False, False, False, False, False, 
        Try 9: True, True, True, True, True, 
        Try 10: True, True, True, True, True, 


And now with mitigated noise.

In [10]:
b = twobit()
for trial in range(1,11):
    message = "        Try " + str(trial)+": "
    b.prepare({'Z':True})
    for repeat in range(5):
        message += str( b.X_value(noisy=True) ) + ", "
    print(message)

        Try 1: True, True, True, True, True, 
        Try 2: False, False, False, False, False, 
        Try 3: True, True, True, True, True, 
        Try 4: False, False, False, False, False, 
        Try 5: True, True, True, True, True, 
        Try 6: True, True, True, True, True, 
        Try 7: False, False, False, False, False, 
        Try 8: False, False, False, False, False, 
        Try 9: True, True, True, True, True, 
        Try 10: False, False, False, False, False, 


In the game [Battleships with complementary measurements](https://medium.com/@decodoku/how-to-program-a-quantum-computer-part-2-f0d3eee872fe), this behaviour is used to implement the attacks that can be made on ships. There are two kinds of attack, with correspond calling `value()` with either `Z` or `X`.

A ship is destroyed when the result is `True`. If `False`, the ship survives the attack. It also becomes immune to it, since another identical call to `value()` will give the same result. Sof fo any hope of succes, the other type of attack must be used. If the ship again survives, it will have forgotten its immunity to the previous attack type. So switching between attacks will ensure victory in the end.

*Note: The following cell is interactive so you'll need to run it yourself*

In [ ]:
ship = twobit()

destroyed = False
while not destroyed:
    basis = input('\n    > Choose a torpedo type (Z or X)...\n    ')
    destroyed = ship.value(basis)
    if destroyed:
        print('\n    *Ship destroyed!*')
    else:
        print('\n    *Attack failed!*')
print('\n    **Mission complete!**')

Note that the `prepare()` method was never called above. This resulting in the first use of `value()` giving a random outcome, regardless of whether `X` or `Z` as used. Also, rather than using the `X_value()` or `Z_value()` methods, we used `value()` with a kwarg to specify `X` or `Z`. This is just a shortcut provided to write nicer programs in cases like these.



Finally, it should be noted that there is a third basis alongside `X` and `Z`. As you could probably guess, it's called `Y`. It is also fully functional, and is even used in the case where `prepare()` is not called to provide an initial state that is random for both `X` and `Z`. So this object should really be called `three_bit`. But it's not, because everyone always ignores poor `Y`.